In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, RobustScaler, MinMaxScaler, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

c:\users\pavlo.cherneha\appdata\local\programs\python\python38\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\users\pavlo.cherneha\appdata\local\programs\python\python38\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
c:\users\pavlo.cherneha\appdata\local\programs\python\python38\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [5]:
df=pd.read_csv("D:\\Guides\\1\\Vacancies\\Baku ML course DAN IT\\Lectures\\Jupyter notebooks\\Credit Risk.csv")
print(df.shape)
df.head()

(614, 13)


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [6]:
X = df[['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed',
        'Loan_Amount_Term', 'Property_Area', 'ApplicantIncome', 'CoapplicantIncome',
        'LoanAmount', 'Credit_History']]
y = df['Loan_Status']


X_train, X_test, y_train, y_test = train_test_split(X, y)

In [7]:
categorical_preprocessing = Pipeline(steps=[
    ('imputation', SimpleImputer(strategy='most_frequent')),
    ('ohe', OneHotEncoder(handle_unknown='ignore'))
])

numeric_preprocessing = Pipeline(steps=[
    ('imputation', SimpleImputer(strategy='median')),
    ('scaling', RobustScaler())
])

preprocessing = ColumnTransformer(transformers=[
    ('categorical', categorical_preprocessing, ['Gender', 'Married', 'Dependents', 'Education',
                                                'Self_Employed',
                                                'Loan_Amount_Term', 'Property_Area']),
    ('numeric', numeric_preprocessing, ['ApplicantIncome', 'CoapplicantIncome',
                                        'LoanAmount', 'Credit_History'])
])

clf = Pipeline(steps=[
    ('preprocessing', preprocessing),
    ('clf', KNeighborsClassifier(n_neighbors=7))
])

In [8]:
y.value_counts()

Y    422
N    192
Name: Loan_Status, dtype: int64

In [12]:
192/(422+192)

0.3127035830618892

In [13]:
from sklearn.metrics import f1_score, make_scorer

In [16]:
cross_val_score(clf, X_train, y_train, cv=5, scoring=make_scorer(f1_score, pos_label="N")).mean()

0.23082670906200317

In [17]:
clf.fit(X_train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('imputation',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Gender', 'Married',
                                                   'Dependents', 'Education',
                                                   'Self_Employed',
                                                   'Loan_Amount_Term',
                                                   'Property_Area']),
                                                 ('numeric',
                                                  Pipeline(steps=[('imputation',
                          

In [18]:
f1_score(y_test, clf.predict(X_test), pos_label='N')

0.24137931034482765

In [20]:
import optuna

In [22]:
def objective(trial):
    n_neighbors = trial.suggest_int('n_neighbors', 3, 101)
    categorical_preprocessing = Pipeline(steps=[
        ('imputation', SimpleImputer(strategy='most_frequent')),
        ('ohe', OneHotEncoder(handle_unknown='ignore'))
    ])

    numeric_preprocessing = Pipeline(steps=[
        ('imputation', SimpleImputer(strategy='median')),
        ('scaling', RobustScaler())
    ])

    preprocessing = ColumnTransformer(transformers=[
        ('categorical', categorical_preprocessing, ['Gender', 'Married', 'Dependents', 'Education',
                                                    'Self_Employed',
                                                    'Loan_Amount_Term', 'Property_Area']),
        ('numeric', numeric_preprocessing, ['ApplicantIncome', 'CoapplicantIncome',
                                            'LoanAmount', 'Credit_History'])
    ])

    clf = Pipeline(steps=[
        ('preprocessing', preprocessing),
        ('clf', KNeighborsClassifier(n_neighbors=n_neighbors))
    ])
    
    return cross_val_score(clf, X_train, y_train, cv=5, scoring=make_scorer(f1_score, pos_label='N')).mean()

In [23]:
study = optuna.create_study(direction='maximize')

[I 2022-02-05 11:42:03,337] A new study created in memory with name: no-name-d540936c-7ad5-49b3-938e-a26b066fc5fb


In [24]:
study.optimize(objective, n_trials=100)

[I 2022-02-05 11:45:50,909] Trial 0 finished with value: 0.0 and parameters: {'n_neighbors': 99}. Best is trial 0 with value: 0.0.
[I 2022-02-05 11:45:51,111] Trial 1 finished with value: 0.07429130009775171 and parameters: {'n_neighbors': 20}. Best is trial 1 with value: 0.07429130009775171.
[I 2022-02-05 11:45:51,310] Trial 2 finished with value: 0.03790322580645161 and parameters: {'n_neighbors': 27}. Best is trial 1 with value: 0.07429130009775171.
[I 2022-02-05 11:45:51,483] Trial 3 finished with value: 0.06290322580645161 and parameters: {'n_neighbors': 26}. Best is trial 1 with value: 0.07429130009775171.
[I 2022-02-05 11:45:51,675] Trial 4 finished with value: 0.0 and parameters: {'n_neighbors': 72}. Best is trial 1 with value: 0.07429130009775171.
[I 2022-02-05 11:45:51,905] Trial 5 finished with value: 0.0 and parameters: {'n_neighbors': 84}. Best is trial 1 with value: 0.07429130009775171.
[I 2022-02-05 11:45:52,077] Trial 6 finished with value: 0.05040322580645161 and param

In [25]:
study.best_params

{'n_neighbors': 4}

In [26]:
clf = Pipeline(steps=[
    ('preprocessing', preprocessing),
    ('clf', KNeighborsClassifier(n_neighbors=4))
])

In [27]:
cross_val_score(clf, X_train, y_train, cv=5, scoring=make_scorer(f1_score, pos_label='N')).mean()

0.3982684665646127

In [28]:
clf.fit(X_train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('imputation',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Gender', 'Married',
                                                   'Dependents', 'Education',
                                                   'Self_Employed',
                                                   'Loan_Amount_Term',
                                                   'Property_Area']),
                                                 ('numeric',
                                                  Pipeline(steps=[('imputation',
                          

In [29]:
f1_score(y_test, clf.predict(X_test), pos_label='N')

0.41237113402061853

In [31]:
def make_clf(n_neighbors, numeric_imputation, scaling_strategy, knn_weights):
    categorical_preprocessing = Pipeline(steps=[
        ('imputation', SimpleImputer(strategy='most_frequent')),
        ('ohe', OneHotEncoder(handle_unknown='ignore'))
    ])

    if scaling_strategy == 'robust':
        scaling = RobustScaler()
    elif scaling_strategy == 'minmax':
        scaling = MinMaxScaler()
    elif scaling_strategy == 'standard':
        scaling = StandardScaler()
        
    numeric_preprocessing = Pipeline(steps=[
        ('imputation', SimpleImputer(strategy=numeric_imputation)),
        ('scaling', scaling)
    ])

    preprocessing = ColumnTransformer(transformers=[
        ('categorical', categorical_preprocessing, ['Gender', 'Married', 'Dependents', 'Education',
                                                    'Self_Employed',
                                                    'Loan_Amount_Term', 'Property_Area']),
        ('numeric', numeric_preprocessing, ['ApplicantIncome', 'CoapplicantIncome',
                                            'LoanAmount', 'Credit_History'])
    ])

    clf = Pipeline(steps=[
        ('preprocessing', preprocessing),
        ('clf', KNeighborsClassifier(n_neighbors=n_neighbors, weights=knn_weights))
    ])
    
    return clf

In [32]:
clf = make_clf(4, 'median', 'robust', 'uniform')

In [33]:
cross_val_score(clf, X_train, y_train, cv=5, scoring=make_scorer(f1_score, pos_label='N')).mean()

0.3982684665646127

In [34]:
clf.fit(X_train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('imputation',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Gender', 'Married',
                                                   'Dependents', 'Education',
                                                   'Self_Employed',
                                                   'Loan_Amount_Term',
                                                   'Property_Area']),
                                                 ('numeric',
                                                  Pipeline(steps=[('imputation',
                          

In [35]:
cross_val_score(clf, X_test, y_test, cv=5, scoring=make_scorer(f1_score, pos_label='N')).mean()

0.3483660130718954

In [37]:
def objective(trial):
    n_neighbors = trial.suggest_int('n_neighbors', 3, 101)
    numeric_imputation = trial.suggest_categorical('numeric_imputation', [
        'median', 'mean'
    ])
    scaling_strategy = trial.suggest_categorical('scaling_strategy', [
        'robust', 'minmax', 'standard'
    ])
    knn_weights = trial.suggest_categorical('knn_weights', [
        'uniform', 'distance'
    ])
    
    clf = make_clf(n_neighbors, numeric_imputation, scaling_strategy, knn_weights)
    
    return cross_val_score(clf, X_train, y_train, cv=5, scoring=make_scorer(f1_score, pos_label='N')).mean()

In [38]:
study = optuna.create_study(direction='maximize')

[I 2022-02-05 12:02:23,061] A new study created in memory with name: no-name-312301aa-fc9a-4e3e-b0a2-710a71c174b7


In [39]:
study.optimize(objective, n_trials=100)

[I 2022-02-05 12:02:42,946] Trial 0 finished with value: 0.5392902446390819 and parameters: {'n_neighbors': 17, 'numeric_imputation': 'mean', 'scaling_strategy': 'standard', 'knn_weights': 'uniform'}. Best is trial 0 with value: 0.5392902446390819.
[I 2022-02-05 12:02:43,164] Trial 1 finished with value: 0.0 and parameters: {'n_neighbors': 52, 'numeric_imputation': 'mean', 'scaling_strategy': 'robust', 'knn_weights': 'uniform'}. Best is trial 0 with value: 0.5392902446390819.
[I 2022-02-05 12:02:43,276] Trial 2 finished with value: 0.5458441558441559 and parameters: {'n_neighbors': 41, 'numeric_imputation': 'mean', 'scaling_strategy': 'standard', 'knn_weights': 'distance'}. Best is trial 2 with value: 0.5458441558441559.
[I 2022-02-05 12:02:43,486] Trial 3 finished with value: 0.5458441558441559 and parameters: {'n_neighbors': 35, 'numeric_imputation': 'mean', 'scaling_strategy': 'standard', 'knn_weights': 'uniform'}. Best is trial 2 with value: 0.5458441558441559.
[I 2022-02-05 12:02:

In [40]:
study.best_params

{'n_neighbors': 66,
 'numeric_imputation': 'mean',
 'scaling_strategy': 'standard',
 'knn_weights': 'distance'}

In [41]:
clf = make_clf(66, 'mean', 'standard', 'distance')

In [42]:
clf.fit(X_train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('imputation',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Gender', 'Married',
                                                   'Dependents', 'Education',
                                                   'Self_Employed',
                                                   'Loan_Amount_Term',
                                                   'Property_Area']),
                                                 ('numeric',
                                                  Pipeline(steps=[('imputation',
                          

In [43]:
f1_score(y_test, clf.predict(X_test), pos_label='N')

0.6865671641791045